In this notebook I am exporting all of the player Fanduel data for the 2020 NBA season. This information was pulled from this site http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=10&day=22&year=2019 using Beautiful soup and then by iterating over the dropdown menu.

While I want to do additional investigation by cleaning the table up to include stats, this first go around was to prove out that I could pull the records for each night of games and then merge them by date and by player to be able to look at the aggregate information for Fanduel points scored vs fanduel price, with some other pieces of information such as team, opponent, and whether the player was on the road vs at home.

In [1]:
#pulling all relevant libraries 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time

the below information was used to scrape the site. I ran this once and then commneted it out after saving the tables as a csv to avoid constantly pulling the data from the web.

In [2]:
##realized the url just updated based on dates, so wrote a loop to accumulate all of the urls
#years = [2019,2020]
#months = list(range(1,13,1))
#days = list(range(1,32,1))

#urls = []
#for year in years:
#    for month in months:
#        for day in days:
#            urls.append(str('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon='+str(month)+'&day='+str(day)+'&year='+str(year)))

##The 2020 season kicked off October 22, 2019 so I want to have a list with just the information for those dates
##finding index number of url that matches that day
#urls.index('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=10&day=22&year=2019')
##ended up being 300
##finding index that matches last game of regular season - March 11. I am not counting playoffs or post pandemic games sense not all teams are playing 
#urls.index('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=3&day=10&year=2020')
##ended up being 443
##saving list just for dates that would be in 2020 season
#seasonurls = urls[300:444]

##we should now have the page we need for each url

This is the script to scrape all of the player data. I only ran this once and saved it as a csv in the section in the code marked below because running it is very time consuming.

In [3]:
##creating an empty list that will have the player row for each table

#rows = []

##Running a loop through all the urls to get every piece of data in one list
#scraping each url
#for url in urls:
#    webpage = requests.get(url)
#    webpage_content = webpage.content
#    soup = BeautifulSoup(webpage_content,'html.parser')
#    table_rows = soup.find_all('td')
    #pulling just the player data, which starts on the 24th entry. 
    #We also want to add some kind of date identifier so for now im doing the url
#    for row in table_rows[24:]:
#        rows.append([row.get_text(),url])

In [4]:
##create data frame
#sample_frame = pd.DataFrame.from_records(rows).reset_index()
##save to csv
#sample_frame.to_csv('All_rows.csv')

The above text can be uncommented if you ever need to pull the information again, for example in 2021. The below information is now pulling from the "All_rows" CSV I saved, which has all data from the beginning of the 2021 season through March 10, the day before games were halted due to COVID.<br><br>
From here on, I'm going to approach the problem using that static csv, starting with creating a data frame

In [5]:
#create frame from saved csv
new_frame = pd.read_csv('All_rows.csv')
#rename columns
new_frame.columns = ['Row','ID','Data','URL']

In [6]:
#clean up column with urls so it just has date
dates = new_frame['URL'].map(lambda x: x.replace('http://rotoguru1.com/cgi-bin/hyday.pl?game=fd&mon=','')\
                             .replace('&year=','/')\
                             .replace('&day=','/'))
new_frame['Date'] = dates

#create separate frame that removes all the url columns
date_frame = new_frame[['Data','Date']].reset_index()
len(date_frame)

580661

Everything from here to the next markup is me clearing out rows that were in the table but did not have player information.

In [7]:
#We need to remove some ads and nav stuff here by 
#converting to a series, finding the ones that match, and adding back to the table
find_Ads = date_frame['Data']
#create series that has 0 for what matches the ads
ads_found = find_Ads.str.find('RotoGuru')
#add column to table with 0's
date_frame['Ad'] = ads_found
#create new table with those rows with zero gone
no_ads = date_frame[date_frame['Ad'] != 1].reset_index()
len(no_ads)

580320

In [8]:
#removing index columns
del no_ads['index']
del no_ads['level_0']

In [9]:
#Repeating to remove Jump To:
#converting to a series, finding the ones that match, and adding back to the table
find_jump = no_ads['Data']
#create series that has 0 for what matches the ads
jump_found = find_jump.str.find('Jump to:')
#add column to table with 0's
no_ads['Remove'] = jump_found
#create new table with those rows with zero gone
jump_gone = no_ads[no_ads['Remove'] !=0].reset_index()
len(jump_gone)

579475

In [10]:
##There's a subtable headers that aren't player data. we are getting rid of most those here. Min can't go 
#because it lines up with al-farouq aminu
##creates list of all the words I want to find and get rid of
sub = ['Forward','Center','FD Points','Salary','Team','Opp.','Score','Stats']
pattern = '|'.join(sub)

jump_gone['gone'] = jump_gone['Data'].str.contains(pattern, case=False)


In [11]:
#clearing columns
del jump_gone['index']
del jump_gone['Remove']

In [12]:
cleaner_table = jump_gone[jump_gone['gone'] != True].reset_index()
#specifically remove Min
#converting to a series, finding the ones that match, and adding back to the table
find_min = cleaner_table['Data']
#create series that has 0 for what matches the word Min
min_found = find_min.str.find('Min')
#add column to table with 0's
cleaner_table['Remove'] = min_found
#create new table with those rows with zero gone
clean_table = cleaner_table[cleaner_table['Remove'] !=0].reset_index()

In [13]:
#removing extra rows
del clean_table['index']
del clean_table['Ad']
del clean_table['gone']
del clean_table['Remove']



Now that we've cleared all the random stuff out, we're going to try to create a row per player per game and include the date played

In [14]:
##create series with the data
just_data = clean_table[['Data','Date']]

In [15]:
 #clearing out a row with an "unlisted" entry
just_data.drop([4158])

Data       Date
0                             SG   1/1/2019
1                 Beasley, Malik   1/1/2019
2                           40.7   1/1/2019
3                         $4,500   1/1/2019
4                            den   1/1/2019
...                          ...        ...
572947                       den  9/22/2020
572948                     v lal  9/22/2020
572949                   114-106  9/22/2020
572950                      4:35  9/22/2020
572951     0pt 2rb 1as 1to 0-1fg  9/22/2020

[572951 rows x 2 columns]

Now we have a frame with every piece of information and a date, but we need to make it one row per player per game

In [16]:
#merging data and date in a column so
#I can then hopefully turn each one into a series and then just have the date once at the end.
just_data['merge_date'] = just_data['Data'].astype(str)+'|'+just_data['Date']
just_data

Data       Date  \
0                             SG   1/1/2019   
1                 Beasley, Malik   1/1/2019   
2                           40.7   1/1/2019   
3                         $4,500   1/1/2019   
4                            den   1/1/2019   
...                          ...        ...   
572947                       den  9/22/2020   
572948                     v lal  9/22/2020   
572949                   114-106  9/22/2020   
572950                      4:35  9/22/2020   
572951     0pt 2rb 1as 1to 0-1fg  9/22/2020   

                                merge_date  
0                              SG|1/1/2019  
1                  Beasley, Malik|1/1/2019  
2                            40.7|1/1/2019  
3                          $4,500|1/1/2019  
4                             den|1/1/2019  
...                                    ...  
572947                       den|9/22/2020  
572948                     v lal|9/22/2020  
572949                   114-106|9/22/2020  
572950                      4:35|9/22/2020  
572951     0pt 2rb 1as 1to 0-1fg|9/22/2020  

[572952 rows x 3 columns]

In [17]:
#turning my merged column into a list so I can run a comprehension and then add the date to the end of a player row
just_datas = just_data['merge_date']
data_list = list(just_datas)


In [18]:
#This is a thing that let you make a row per player, but you couldn't add the date
##each row was 9 entries. So we're gonna write this thing to create a series with lists of 9 entries each. This gets thrown off very easily though
players = [data_list[x:x+9] for x in range(0, len(data_list), 9)]


In [19]:
#now we have a row per player, but every piece of data has the date. So we want to just make that its own entry at the end
for player in players:
    player.append(player[0].split('|')[1])
players

[['SG|1/1/2019',
  'Beasley, Malik|1/1/2019',
  '40.7|1/1/2019',
  '$4,500|1/1/2019',
  'den|1/1/2019',
  'v nyk|1/1/2019',
  '\xa0115-108|1/1/2019',
  '29:56|1/1/2019',
  ' \xa0 23pt 6rb 5as 1st 5trey 8-15fg 2-2ft|1/1/2019',
  '1/1/2019'],
 ['SG|1/1/2019',
  'Hield, Buddy^|1/1/2019',
  '40|1/1/2019',
  '$6,900|1/1/2019',
  'sac|1/1/2019',
  'v por|1/1/2019',
  '\xa0108-113|1/1/2019',
  '39:02|1/1/2019',
  ' \xa0 27pt 5rb 4as 1bl 2to 5trey 11-23fg|1/1/2019',
  '1/1/2019'],
 ['PG|1/1/2019',
  'Lillard, Damian^|1/1/2019',
  '39.2|1/1/2019',
  '$9,200|1/1/2019',
  'por|1/1/2019',
  '@ sac|1/1/2019',
  '\xa0113-108|1/1/2019',
  '41:54|1/1/2019',
  ' \xa0 25pt 6rb 6as 1st 5to 1trey 8-21fg 8-8ft|1/1/2019',
  '1/1/2019'],
 ['PG|1/1/2019',
  'Simmons, Ben^|1/1/2019',
  '38.8|1/1/2019',
  '$9,800|1/1/2019',
  'phi|1/1/2019',
  '@ lac|1/1/2019',
  '\xa0119-113|1/1/2019',
  '32:50|1/1/2019',
  ' \xa0 14pt 9rb 8as 1st 1bl 4to 6-13fg 2-6ft|1/1/2019',
  '1/1/2019'],
 ['SG|1/1/2019',
  'Bogdanovic, B

In [24]:
players[4][7]

'35:33|1/1/2019'

In [21]:
player_rows =[]
for player in players:
    player_rows.append(player[0].split('|')[0])
    player_rows.append(player[1].split('|')[0])
    player_rows.append(player[2].split('|')[0])
    player_rows.append(player[3].split('|')[0])
    player_rows.append(player[4].split('|')[0])
    player_rows.append(player[5].split('|')[0])
    player_rows.append(player[6].split('|')[0])
    player_rows.append(player[7].split('|')[0])
    player_rows.append(player[7].split('|')[0])
    player_rows.append(player[9])

IndexError: list index out of range